# 新聞爬蟲：Reuters China-US Trade War

先以selenium爬取新聞網站，再用BeautifulSoup解析html碼，得到日期、標題、摘要以及內文，最後存成pickle檔

In [13]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from bs4 import BeautifulSoup
import ssl
import datetime
import requests

In [15]:
driver = webdriver.Chrome("/Users/gillian0531/DS_jupyter/chromedriver")
#driver.get("https://www.bbc.com/news/topics/cwlw3xz0lvvt/brexit")
driver.get("https://www.reuters.com/search/news?blob=china-us+trade+war")


more_path = '//*[@id="content"]/section[2]/div/div[1]/div[4]/div/div[4]/div[1]' # find click button path
context = ssl._create_unverified_context()


for i in range(400):
    
    elementToClick = driver.find_element_by_xpath(more_path)
    #print(elementToClick.location['y'])
    driver.execute_script("window.scrollTo(0," + str(elementToClick.location['x']) +")")
    elementToClick.click()
    driver.implicitly_wait(5)
    print(i)
pageSource = driver.page_source
bsObj = BeautifulSoup(pageSource)

NoSuchWindowException: Message: no such window: window was already closed
  (Session info: chrome=74.0.3729.157)
  (Driver info: chromedriver=74.0.3729.6 (255758eccf3d244491b8a1317aa76e1ce10d57e9-refs/branch-heads/3729@{#29}),platform=Mac OS X 10.14.4 x86_64)


In [12]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.86 Safari/537.36'}
proxies = ""

try:
    articles = bsObj.find_all("div", {"class" : "search-result-indiv"})
except:
    pass

cnt = 0
news_data = []
for news in articles:
    date = news.find("h5", {"class" : "search-result-timestamp"}).get_text()[:-6] # type: str
    dates = datetime.datetime.strptime(date, "%B %d, %Y %I:%M")
    title = news.find("h3", {"class" : "search-result-title"}).get_text()
    summary = news.find("div", {"class" : "search-result-excerpt"}).get_text()
    links = 'http://www.reuters.com'+(news.find('a')['href'])
    html = links
    #print(html)
    re = requests.get(html, headers = headers, proxies = proxies)
    bs = BeautifulSoup(re.content, 'lxml')
    contents= []
    for p in bs.select('.StandardArticleBody_body p')[:]:
        contents.append(p.text.strip())
    article = ' '.join(contents)
    #print(article)
    if article != '':
        news_data.append({
        'date': dates,
        'title' : title,
        'summary' : summary,
        'content' : article})
    print(cnt)
    cnt += 1
print(len(news_data))


NameError: name 'articles' is not defined

In [33]:
import pandas as pd
pd.DataFrame(news_data)[['date', 'title', 'summary', 'content']].head()

,date,title,summary,content
0,2018-09-18 08:24:00,China will use trade war with US to replace im...,"... measures"" the United States is taking in t...","SHANGHAI, Sept 19 (Reuters) - China is not afr..."
1,2019-05-13 04:28:00,US STOCKS-Wall St shudders as U.S.-China trade...,"... belligerent\ntrade war, sending investors ...",NEW YORK (Reuters) - Wall Street sank on Monda...
2,2019-05-13 02:38:00,US STOCKS-Wall St dives as U.S.-China trade wa...,.../ in a news window)\n* China to impose tari...,"(For a live blog on the U.S. stock market, cli..."
3,2018-10-12 04:02:00,Lenders brace for trade war,"...-China trade war intensifies, dealing a\nfu...",* Loans: Business from China to take further h...
4,2018-09-07 03:07:00,"Australian, NZ shares fall on worries US-China...","..., partly because of the\npossibility the U....",* Australian shares have worst week since Febr...


In [34]:
import pickle

file = open('Reuters_news.pickle', 'wb')
pickle.dump(news_data, file)
file.close()